# A GUIDE INTO MLOPS

MLOps is one of the crucial phase in a machine learning lifecycle to keep machine learning models maintained and perform well. There's an open source MLOps tool that easily to use, comprehensive ML library, and manage end-to-end ML workkflows from development to production. It is called [MLflow](https://mlflow.org). Here's how to understand by doing a fundamental implementation using ML flow. Let's just dive into the notebook.

# 1. Install MLflow

run it on your IDE terminal

`pip install mlflow`

# 2. Launch the Mlflow Tracking Server

In [2]:
import mlflow
from mlflow.entities import experiment
from mlflow.models import infer_signature
from mlflow import MlflowClient
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [3]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

In [4]:
all_experiments = client.search_experiments()
print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1736852050555, experiment_id='0', last_update_time=1736852050555, lifecycle_stage='active', name='Default', tags={}>]


In [8]:
default_experiment = [
    {
        "name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage
    }
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


## Creating experiments

In [9]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the grocery forecasting project. "
    "This experiment contains the produce models for apples."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags=experiment_tags
)


## Search based on tags

### Interactive tables

Effortlessly view, navigate, sort, and filter data. Create charts and access essential data insights, including descriptive statistics and missing values – all without writing a single line of code.

In [ ]:
# Defining data for the dataframe
data = {
    'Basket': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'],
    'Apples': [10, 20, 30, 56, 40, 40, 67, 47, 40, 4, 49, 52, 5, 56, 35, 45],
    'Bananas': [15, 6, 3, 45, 67, 44, 45, 11, 14, 18, 13, 12, 1, 34, 12, 12]
}

# Creating the dataframe
df = pd.DataFrame(data)

df

### Visualization in IDE

Create graphs and visualizations that match your chosen color scheme.

In [ ]:
# Calculate the sums
sum_apples = df['Apples'].sum()
sum_bananas = df['Bananas'].sum()

# Create a bar chart
plt.bar(['Apples', 'Bananas'], [sum_apples, sum_bananas], color=['red', 'blue'])

# Set a title
plt.title('Comparison of total Apples and Bananas')

# Show the plot
plt.show()